## Anna Kaniowska - Cellphones & Accessories dataset analysis

The goal of this project is to extract as much information form the data set that can be obtained here - http://snap.stanford.edu/data/amazon/Cell_Phones_&_Accessories.txt.gz (source: http://snap.stanford.edu/data/web-Amazon-links.html)

In [ ]:
# All imports needed to perform the analysis
import gzip
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, silhouette_score
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.metrics.pairwise import cosine_similarity
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from sklearn.decomposition import PCA
from wordcloud import WordCloud

#### Loading the dataset

In [ ]:
# A modified version of function available on the source page
def parse(filename):
    """
    Parses a gzipped text file and returns a list of dictionaries containing the parsed data.

    Parameters:
    filename (str): The path to the gzipped text file to be parsed.

    Returns:
    list: A list of dictionaries containing the parsed data.
    """
    f = gzip.open(filename, 'rb')
    entry = {}
    data = []
    for line in f:
        l = line.decode('utf-8').strip()
        colonPos = l.find(':')
        if colonPos == -1:
            data.append(entry)
            entry = {}
            continue
        eName = l[:colonPos]
        rest = l[colonPos+2:]
        entry[eName] = rest
    data.append(entry)
    return data

# Loading the dataset
data = parse('Cell_Phones_&_Accessories.txt.gz')
df = pd.DataFrame(data)


#### Getting to know the dataset

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
print(f"The shape of the dataset is: {df.shape}")

Taking a first look at the data, it is visible that it shows the reviews that customers gave to the products. The products are mainly cellphones and their accesories. The dataset is big - almost 79 000 rows is a significant amount of data. 10 columns provide information about the rated product, the customer and their opinion on the product. When it comes to technical details - it is necessary to change 'unknown' values to NaN in order to prepare data to further analysis. Checking the dataset for duplicated rows and dropping existing ones is also necessary because this is something that cannot be seen at first glance. Another conclusion is that the types of the columns are not necessarily correct (e.g. product/price should be stored as float), it is also needed to be corrected.

## Feature Engineering & Exploratory Data Analysis

#### Checking for duplicated rows

In [ ]:
print(f"There is {df.duplicated().sum()} duplicated rows in the dataset")

In [ ]:
df.drop_duplicates(inplace=True)

#### Handling missing values

In [ ]:
df.replace("unknown", np.nan, inplace=True)

# Analyzing the missing values occurences
print("Missing values occurences:")
print(df.isna().sum())

# Checking anonymous reviews (those where userId and profileName is missing)
anon_reviews_perc = df['review/userId'].isna().sum()/df.shape[0] * 100
print(f"{anon_reviews_perc:.2f}% of the reviews are anonymously submitted.")


The first conclusion is one row that can be safely deleted in each column (it is very likely that it is the same row for each of the columns). \
The second conclusion refers to anonymous reviews. When missing values are less than 5% of given feature, they can be safely deleted without having impact on further analysis. For now the data will be divided into to sets - first with anonymous reviews and second with named reviews. It may be useful to further analysis.

In [ ]:
# Extracting columns with 1 missing value
cols = ['product/productId', 'product/title', 'review/helpfulness', 'review/score', 'review/time', \
        'review/summary', 'review/text']
df = df.dropna(subset=cols)
print(f"Shape of the dataset after dropping NaNs: {df.shape}")

As expected, only one row of the data was deleted.

Before dividing dataset into two separate dataset it would be useful to correct the columns' types.

#### Correcting columns types

In [ ]:
df['product/price'] = pd.to_numeric(df['product/price'], errors='coerce')
df['review/score'] = pd.to_numeric(df['review/score'], errors='coerce')
df['review/time'] = pd.to_datetime(df['review/time'].astype(float), unit='s')

def handle_helpfulness(x):
    """
    Converts the string representation of helpfulness scores to a float value between 0 and 1.

    Parameters:
        helpfulness (str): The string representation of helpfulness scores, in the format "x/y",
        where "x" is the number of users who found the review helpful and "y" is the total number of votes.

    Returns:
        float: The float value of the helpfulness score, calculated as "x / y". Returns 0 if "y" is 0.
    """
    try:
        nom, denom = x.split("/")
        return int(nom) / int(denom)
    except (ValueError, ZeroDivisionError):
        return 0

df['review/helpfulness'] = df['review/helpfulness'].apply(handle_helpfulness)

The price and score columns are stroing numeric values, the review has time in seconds. When it comes to helpfulness it was transformed to the float value that represents it.

Coming back to the missing values before the set division, the last thing about them is the price. As it is a significant amount of data in the dataset it would not make sense to drop it. Taking into consideration that assigning a price of a small accessory to a brand new cellphone would distort the dataset, the missing values in this column will not be replaced with mean, median or mode. The products are stored in more or less an order (similiar products next to one another) so an optimal way to impute the missing values would be kNN method.

In [ ]:
features_with_missing_values = df[['product/price']]
imputer = KNNImputer(n_neighbors=5)
imputed_features = imputer.fit_transform(features_with_missing_values)
df['product/price'] = imputed_features

#### Division of a dataset

In [ ]:
# Dividing the dataset into anonymous and named
df_anon = df[(df['review/userId'].isna()) & (df['review/profileName'].isna())]
df_named = df[~(df['review/userId'].isna()) & ~(df['review/profileName'].isna())]

# Deleting unnecessary columns from the anonymous reviews
df_anon = df_anon.drop(columns=['review/userId', 'review/profileName'])

# Resetting index
df_named.reset_index(inplace=True, drop=True)
df_anon.reset_index(inplace=True, drop=True)

# Checking if everything went as expected
print(f"Missing values occurences in anonymous reviews dataset:\n{df_anon.isna().sum()}")
print(f"Missing values occurences in named reviews dataset:\n{df_named.isna().sum()}")

The analysis will focus on the named reviews but the anonymous ones will always be an available point of reference. This decision is supported by the fact that named reviews can be treated as more reliable.

#### Data Visualizations

In [ ]:
# Taking a look on the prepared data
df_named.head()

In [ ]:
df_anon.head()

In [ ]:
# Extracting the columns with numercial and categorical variables
numerical = ['product/price', 'review/helpfulness', 'review/score', 'review/time']
categorical = ['product/productId', 'product/title', 'review/userId', 'review/profileName']

Summary and text would be difficult to visualize so those columns are not taken into consideration in this section.

#### a. Named Reviews

In [ ]:
# Numerical variables
fig, axs = plt.subplots(2, 2, figsize=[15, 15])
fig.suptitle("Data distribution for Named Reviews")

max_counts = [df_named[col].value_counts().max() for col in numerical]
for i, col in enumerate(df_named[numerical]):
    sns.histplot(data=df_named, x=col, ax=axs[i//2, i%2], color='darkmagenta')
    axs[i//2, i%2].set(title=col, xlabel='Value')
plt.show()

It is visible that most of the reviews are rather extremely helpful or extremely unhelpful, something in between is not seen very often. A similiar situation can be observed in the Score histogram but not to such extent (4.0 note is observed more often than 1.0). Majority of reviews has a score 5.0 which is a sign of good quality of the products. \ 
What is interesting is that most of the reviews were registered in 2007-2008 which is the time of Global Financial Crisis. It suggests that during this time people were carefully watching their expenses and looking and the quality of products more than usual. This could lead to the highest amount of the reviews on the plot. There is another theory why this peak took place. In 2007 Amazon experienced servers outages that caused about 2 hours of downtime on the platform. It was a famous case, so when people talked about, they might have decided to use their services and write some reviews. The topics are not directly related but they both refer to Amazon websites so there might be a correlation. \
The plot which refers to the price is not very transparent, so in order to better understand the data there was a boxplot created.

In [ ]:
fig, ax = plt.subplots(figsize=[8,6])
sns.boxplot(x=df_named['product/price'], ax=ax, color='darkmagenta')
ax.set(xlabel='Product Price', title="Boxplot for Product Price (Named Reviews)")
plt.show()

It is important to remember that more than a half of the price values were imputed by kNN algorithm so the results might not be 100% reliable. That is also the reason why outliers are not removed. Majority of the product is in the cheaper section, while more expensive products are bought less often - they are treated as outliers in this dataset. Half of the dataset (between first and third quartile) is represented by (15,35) range (approximately).

#### b. Anonymous Reviews

In [ ]:
# Numerical variables
fig, axs = plt.subplots(2, 2, figsize=[15, 15])
fig.suptitle("Data distribution for Anonymous Reviews")

max_counts = [df_anon[col].value_counts().max() for col in numerical]
for i, col in enumerate(df_anon[numerical]):
    sns.histplot(data=df_anon, x=col, ax=axs[i//2, i%2], color='darkmagenta')
    axs[i//2, i%2].set(title=col, xlabel='Value')
plt.show()

The results are similar to those from Names Reviews dataset. The biggest difference is visible on the last plot - anonymous reviews were the most popular in 2004. It might prove that people in this time were not trusting the internet as much as they did in 2007-2008 and did not want to provide the personal data to any websites. It is also interesting to see that starting from 2008 the anonymous reviews are rarely observed.

## Machine Learning analysis

In this section there will appear several machine learning techiniques used to get as much information form the data as possible.

#### Natural Language Processing - data preparation

In [ ]:
# Prepare necessary tools
nltk.download('stopwords')
stop_words = stopwords.words('english')
stemmer = SnowballStemmer('english')

def preprocess_text(text):
    """
    Preprocesses a given text by tokenizing it into words, converting all words to lowercase, 
    removing any non-alphabetic characters and stop words, and stemming the remaining words. 
    The processed words are then joined into a single string and returned.

    Args:
    text (str): A string of text to be preprocessed.

    Returns:
    str: The preprocessed text, consisting of stemmed words in lower case, with non-alphabetic characters and stop words removed and all words joined into a single string.
    """
    words = nltk.word_tokenize(text.lower())
    words = [w for w in words if w.isalpha() and w not in stop_words]
    words = [stemmer.stem(w) for w in words]
    return ' '.join(words)

df_named['text'] = df_named['review/summary'].apply(preprocess_text)
df_anon['text'] = df_anon['review/summary'].apply(preprocess_text)


At the beginning of Natural Language Processing it is necessary to prepare the data - handle to stop words, stemming or non-alphabetic characters. The choice between Summary and Text column was not easy, but both of the columns contain natural language, so Summary was picked.

#### Text analysis

In [ ]:
# Creating a bag of words from the preprocessed text
vectorizer = CountVectorizer()
bag_of_words = vectorizer.fit_transform(df_named['text'])

# Extracting the frequency of each word
word_freq = dict(zip(vectorizer.get_feature_names_out(), bag_of_words.sum(axis=0).tolist()[0]))
top_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)[:10]

# Displaying top 10
print("Most common words:")
for word, freq in top_words:
    print(word, freq)

After this simple analysis it is visible that the most frequently used words are adjectives and adverbs like great, good, best. However, the words describing products like phone, case, headset or product itself are equally important. This confirms that the reviews are focused on rating the products and are not randomly written sentences. However, there are some typos like 'batteri' in them. It might suggest that not all of the customers are english native speakers which is very possible.

In [ ]:
# Analyzing sentiment using TextBlob and VADER
df_named['polarity'] = df_named['review/text'].apply(lambda x: TextBlob(x).sentiment.polarity)
analyzer = SentimentIntensityAnalyzer()
df_named['vader_scores'] = df_named['review/text'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

# Performing topic modeling using LDA
texts = [doc.split() for doc in df_named['text']]
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
lda_model = LdaModel(corpus, num_topics=5, id2word=dictionary)
topics = lda_model.print_topics()

# Displaying the topics
for topic in topics:
    print(topic)
    
# Visualizing the results
sns.histplot(data=df_named, x='polarity', kde=True, color='magenta')
plt.title("Distribution of sentiment polarity")
plt.show()

sns.histplot(data=df_named, x='vader_scores', kde=True, color='magenta')
plt.title("Distribution of sentiment scores (VADER)")
plt.show()

To get to know the text charasteristics better LDA (Latent Dirichlet Allocation) was used. The model assumes that documents are made up of multiple topics, and each topic is made up of a set of words. The algorithm takes a corpus of documents as input and tries to identify the underlying topics based on the frequency of words within each document. LDA assumes that each document contains a mixture of topics and each word in the document is associated with one of the topics. \
The algorithm shows 5 indentified topics with top 10 words in each of them. The topics are usually based on the reviewed product or some common features of given product. \
The first plot shows sentiment polarity - in this dataset the reviews are rather positive. This is proven by the fact that the plot resembles normal distribution with mean at 0.25 value. Sentiment polarity ranges from -1 (negative sentiment) to 1 (positive sentiment) where 0 is neutral. The peak on 0.0 can be also observed so there is a significant amount of neutral reviews too. \
The second plot show the VADER (Valence Aware Dictionary and sEntiment Reasoner) analysis. It is based on vader score which also ranges from -1 to 1 based on negativity or positivity of analyzed text. This method uses a dictionary with scores for many words and expressions and, what's more an algorithm that takes the context into consideration. It also shows that positive reviews are the majority of the dataset and that neutral reviews are standing out.

#### Sentiment analysis

In [ ]:
# Labeling
df_named['sentiment'] = df_named['review/score'].apply(lambda score: 1 if score > 3 else 0)

# Showing the basic information
positive_reviews = df_named['sentiment'].sum() / len(df_named) * 100
negative_reviews = 100 - positive_reviews
print(f"Percentage of positive reviews: {positive_reviews:.2f}%. Negative reviews: {negative_reviews:.2f}%")

# Preparing training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_named['text'], df_named['sentiment'], test_size=0.2, random_state=42)

# Converting the text data into a numerical representation using TF-IDF
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Training and evaluating a logistic regression model
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}, F1 score: {f1:.2f}')

After sentiment analysis it is really easy to see whether the product has a positive or negative review. The perecentages of positive and negatice reviews are a confirmation for the conclusions from sentiment histograms. This information is stored in the Sentiment column. Based on this column a machine learning model was trained. The model could have been based on various binary classification algorithms such as Decision Trees or SVM, but Logistic Regression was chosen. The evaluation showed that the model works well.

In [ ]:
# sample 33% of the rows randomly
sample_df = df_named.sample(frac=0.33, random_state=42)
sample_df = sample_df.reset_index(drop=True)
sample_df

Because of computational reasons the decision about sampling was made. Some of the next algorithms are going to be executed on a 33% randomly selected rows from the original dataset. Those algorithms should be performed on original dataset if the technical conditions are met. There was an attempt to choose a strategy with dividing the set to batches and then concatenating the results of manipulations on every single batch. However, this also unfortunately did not work.

#### Clustering

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_anon['text'])

# Trying to find optimal hyperparameters for the DBSCAN algorithm
silhouette_scores = []
for eps in np.arange(0.1, 1.0, 0.1):
    for min_samples in range(2, 10):
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        dbscan.fit(X)
        labels = dbscan.labels_
        if np.max(labels) > 0: # ensure there is at least one cluster
            score = silhouette_score(X, labels)
            silhouette_scores.append((eps, min_samples, score))
best_eps, best_min_samples, best_score = max(silhouette_scores, key=lambda x: x[2])
print(f'Best hyperparameters: eps={best_eps}, min_samples={best_min_samples}, score={best_score}')

# Fitting the DBSCAN with the found hyperparameters and showing the clusters
dbscan = DBSCAN(eps=best_eps, min_samples=best_min_samples)
dbscan.fit(X)
labels = dbscan.labels_
n_clusters = len(set(labels)) - (1 if -1 in labels else 0) # count the number of clusters
print(f'Number of clusters: {n_clusters}')
for i in range(n_clusters):
    cluster_idx = labels == i
    cluster_text = df_anon.loc[cluster_idx, 'review/summary']
    print(f'Cluster {i}:')
    print(cluster_text.head(5))


The results of clustering were not very satisfying on the sample_df. There were attempts to use KMeans method and agglomerative clustering, however they ended up with worse results than presented DBSCAN (agglomerative clustering took a lot of time to execute and KMeans achieved the worst silhouette score). That is why the author decided to present clustering on the dataset with anonymous reviews. Similarly to previous attempts there are a lot of clusters created and not all of them are correctly grouped. Also the score is not satisfying. It could be a great start to further analysis but the author does not have the skills yet (but is willing to learn).

#### Recommender system

In [ ]:
vectorizer = TfidfVectorizer()
product_titles = vectorizer.fit_transform(sample_df['product/title'])

# Calculating similarity of the products
cosine_sim_matrix = cosine_similarity(product_titles)

def get_similar_products(product_id, n=5):
    """
    Find the top n products that are most similar to the given product ID based on their titles.

    Parameters
    ----------
    product_id : str
        The product ID to search for.
    n : int, optional
        The number of similar products to return (default is 5).

    Returns
    -------
    pandas.DataFrame
        A DataFrame containing the top n products that are most similar to the given product ID.
        The DataFrame contains two columns: 'product/productId' and 'product/title'.
    """
    unique_products = sample_df.groupby('product/productId').first().reset_index()
    idx = unique_products.index[unique_products['product/productId'] == product_id].tolist()[0]
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_indices = [i for i, _ in sim_scores[1:n+1]]
    
    return unique_products.loc[top_indices, ['product/productId', 'product/title']]

# Example usage on 'B000JIK92C' product
get_similar_products('B000JIK92C')

This is a recommender system that returns top n products that are similar to the given one basen on their titles. This could be very useful to recommend the returned products to the users that bought the one passed to the function. It could increase the sales significantly.

#### Time Series analysis

In [ ]:
df_named.set_index('review/time', inplace=True)
# Resampling the data by month and calculating mean sentiment score and number of reviews
df_monthly = df_named.resample('M').agg({'vader_scores': 'mean', 'review/userId': 'count'})

# Plotting mean sentiment score and number of reviews over time
fig, axs = plt.subplots(figsize=[12,6])
axs1 = axs.twinx()
sns.lineplot(x=df_monthly.index, y=df_monthly['vader_scores'], ax=axs, color='steelblue')
sns.lineplot(x=df_monthly.index, y=df_monthly['review/userId'], ax=axs1, color='magenta')
axs.set(title="Sentiment and Review Trends Over Time", xlim=[df_monthly.index[0], df_monthly.index[-1]], ylim = [0, 1.05], ylabel='Mean Sentiment Score')
axs1.set(ylim=[-100,3000], ylabel='Number of Reviews')
plt.show()

Approaching the data form the other side, a time series is used to show how the amount of reviews and their mean sentiment changed overtime. The most interesting facts are that the peak of reviews amount took place in 2007 and that mean sentiment score has a downward trend. \
At the very beginning the mean sentiment was very unstable but always above 0 which means that the reviews were rather positive the whole time. Later the values happened to be more stabilized and it is clearly visible than the mean drops each year. \
Coming back to the number of reviews, after the peak in 2007 the values also dropped but at the end of the dataset (circa 2013) they peaked again. 2013 was the year when Amazon introduced AmazonFresh (it offers the delivery of groceries including fresh fruits and vegetables). This is not connected to the topic of the dataset but because of this release Amazon was all over the news and it might have reminded people to log in and leave a review or two. However, this is only an uncertain suggestion and can be treated as an interesting fact.

#### DImensionality reduction

In [ ]:
# Making a list of the reviews and vectorizing it using TF-IDF
reviews = df_named['text'].tolist()
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(reviews)

# Reducing the dimensionality using PCA and visualizing results
pca = PCA(n_components=3)
pca_matrix = pca.fit_transform(tfidf_matrix.toarray())
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(pca_matrix[:,0], pca_matrix[:,1], pca_matrix[:,2], alpha=0.2)
ax.set(xlabel='PC1', ylabel='PC2', zlabel='PC3', \
       title="Visualization of Review Data using PCA Dimensionality Reduction")
plt.show()


The PCA algorithm reduces the TF-IDF matrix to 3 dimensions - each representing another variable. \
In this PCA plot the relationships between the variables are showed, each point represents a review, and the position of the point in the plot indicates how similar or dissimilar it is to other reviews. The distance between two points indicates how different their underlying features are. \
It is visible that there are a lot of groups of points clustered together which means that there are sets of similar reviews. Clustering can be done on this data but, as mentioned before, it is not so easy.

#### Visualzations more or less related to previous section

In [ ]:
# Creating wordcloud image based on the reviews
text = " ".join(review for review in df_named['review/text'])
wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(text)
plt.figure(figsize=[10,10])
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

The wordmap shows the most popular words in review and helps to understand the most common topics. The result is not very surprising - the words revolve around cellphones, their accesories and the quality of the products. However, it is useful to see such visualization.

In [ ]:
# Checking correlation between the variables
df_named['review/length'] = df_named['review/text'].apply(len) # Length of review might be an interesting variable
df_heatmap = df_named[['review/length', 'review/helpfulness', 'vader_scores', 'polarity']].copy()
corr_matrix = df_heatmap.corr()
sns.set(style="white")
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap="rocket_r")
plt.title('Correlation Matrix Heatmap')
plt.show()

The heatmap unfortunately does not provide many interesting patterns. The visible correlation is the one between vader scores and polarity - it is known that they are directly related. Unfortunately, there are not any significant correlations beween other variables. Only review helpfulness is moderately correlated to the length of the review - the longer the review is, the more helpful.

## Summary

The dataset provided a lot of information about the cellphones and their accesories reviews submitted on Amazon website. The analyzed reviews were collected between circa 2000 and 2013. At the beginning customers were tending to leave the reviews anonymously but later (2006/2007) that changed. The analysis focused on the reviews that were not anonymous. Overall, the majority of the reviews were positive. Several machine learning alogirthms were used to perform the analysis. In addition to the machine learning algorithms, several exploratory data analysis techniques were applied to gain insights into the data, including sentiment analysis, topic modeling, and dimensionality reduction. The results of the analysis showed that the most commonly mentioned topics in the reviews were related to product features such as battery life, screen size, and sound quality. The dataset can be used to inform product development and marketing strategies for cellphone and accessory manufacturers.